<a href="https://colab.research.google.com/github/Whisperer8526/Kaggle-Competitions/blob/main/Titanic%20model%20testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Upload

In [ ]:
from google.colab import files
t1 = files.upload()

Saving X_train.npy to X_train (1).npy


In [ ]:
t2 = files.upload()

Saving X_test.npy to X_test (1).npy


In [ ]:
t3 = files.upload()

Saving y_train.npy to y_train (1).npy


In [ ]:
import numpy as np

X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_test = np.load("X_test.npy")

In [ ]:
X_train.shape

(891, 27)

In [ ]:
X_test.shape

(418, 27)

In [ ]:
y_train.shape

(891,)

# Fully Connected Neural Network

In [ ]:
from tensorflow import keras
from functools import partial

RegularDense = partial(keras.layers.Dense,
                       activation="elu", 
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l1_l2(0.001))

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(27)))

model.add(keras.layers.BatchNormalization())
model.add(RegularDense(150))

model.add(keras.layers.BatchNormalization())
model.add(RegularDense(150))

model.add(keras.layers.BatchNormalization())
model.add(RegularDense(75))

model.add(keras.layers.BatchNormalization())
model.add(RegularDense(40))

model.add(keras.layers.BatchNormalization())
model.add(RegularDense(20))

model.add(keras.layers.BatchNormalization())
model.add(RegularDense(10))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(1, activation="sigmoid",
                                  kernel_initializer="glorot_uniform"))

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(loss="binary_crossentropy",
             optimizer=Adam(learning_rate=0.001),
             metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=100,
                   batch_size=32, validation_split=0.1, 
                    shuffle=True)

In [ ]:
model_y_pred = model.predict(X_test)

In [ ]:
y_pred = np.rint(model_y_pred).flatten().astype(int)

In [ ]:
import pandas as pd

data = {"PassengerId" : np.arange(892, 1310),
        "Survived" : y_pred}
nn_data = pd.DataFrame(data=data, index=None)

nn_result = nn_data.to_csv("nn_result.csv", index=False)
nn_export = files.download("nn_result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Voting model

SVM Classifier

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 10, 100, 1000], 
              'gamma': [0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
svc_search = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)
svc_search.fit(X_train, y_train)

In [ ]:
print(svc_search.best_estimator_)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='poly', 
              C=0.1,
              gamma=1,
              probability=True,
              verbose=True, 
              decision_function_shape='ovr')
              
svc.fit(X_train, y_train)

[LibSVM]

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='poly',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

In [ ]:
svc_y_pred = svc.predict(X_test)

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {
    'bootstrap': [True],
    'max_depth': [2, 5, 10],
    'max_features': [2, 3],
    'min_samples_leaf': [2, 5],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [20, 50, 100, 250, 500]}
rf_search = GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3)
rf_search.fit(X_train, y_train)

In [ ]:
print(rf_search.best_estimator_)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features=3,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features=3,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
rf_clf.fit(X_train, y_train)


In [ ]:
rf_y_pred = rf_clf.predict(X_test)

Decision Tree

In [ ]:
from sklearn import tree
dt = tree.DecisionTreeClassifier
param_grid = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [2,4,6,8,10,12]}
dt_search = GridSearchCV(dt() ,param_grid,refit=True,verbose=3)
dt_search.fit(X_train, y_train)

In [ ]:
print(dt_search.best_estimator_)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


In [ ]:
dt_clf = tree.DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

dt_clf.fit(X_train, y_train)

In [ ]:
dt_y_pred = dt_clf.predict(X_test)

K Nearest Neightbours Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier
param_grid = {
    'n_neighbors' : [2, 5, 10, 15],
    'weights' : ['uniform', 'distance']}
knn_search = GridSearchCV(knn() ,param_grid,refit=True,verbose=3)
knn_search.fit(X_train, y_train)

In [ ]:
print(knn_search.best_estimator_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')


In [ ]:
knn_clf = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

knn_clf.fit(X_train, y_train)

In [ ]:
knn_y_pred = knn_clf.predict(X_test)

Gaussian Naive Bayes Clasiffier

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()
gnb_clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
gnb_y_pred = gnb_clf.predict(X_test)

SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
param_grid = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
              "penalty": ["l1", "l2"]}

sgd_search = GridSearchCV(sgd ,param_grid,refit=True,verbose=3)
sgd_search.fit(X_train, y_train)

In [ ]:
print(sgd_search.best_estimator_)

SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l1',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)


In [ ]:
sgd_clf = SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l1',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

sgd_clf.fit(X_train, y_train)

In [ ]:
sgd_y_pred = sgd_clf.predict(X_test)

Logistic Regression 

In [ ]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
param_grid = {'C' : np.logspace(-4, 4, 50),
              'penalty' : ['l1', 'l2']}
log_search = GridSearchCV(log ,param_grid,refit=True,verbose=3)
log_search.fit(X_train, y_train)

In [ ]:
print(log_search.best_estimator_)

LogisticRegression(C=1.7575106248547894, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [ ]:
log_clf = LogisticRegression(C=1.7575106248547894, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

log_clf.fit(X_train, y_train)

In [ ]:
log_y_pred = log_clf.predict(X_test)

AdaBoost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
param_grid = {'n_estimators' : [25, 50, 100, 200, 300],
              'learning_rate' : [0.001, 0.01, 0.1, 0.5, 1, 10]}
ada_search = GridSearchCV(ada ,param_grid,refit=True,verbose=3)
ada_search.fit(X_train, y_train)


In [ ]:
print(ada_search.best_estimator_)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
                   n_estimators=200, random_state=None)


In [ ]:
ada_clf = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
                   n_estimators=200, random_state=None)
ada_clf.fit(X_train, y_train)

In [ ]:
ada_y_pred = ada_clf.predict(X_test)

Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier

vot_clf = VotingClassifier(estimators=[("svc", svc), ("rf", rf_clf), 
                                       ("dt", dt_clf), ("knn", knn_clf), ("gnb", gnb_clf),
                                       ("sgd", sgd_clf), ("log", log_clf), ("ada", ada_clf)],
                           voting="hard")

vot_clf = vot_clf.fit(X_train, y_train) 

[LibSVM]

In [ ]:
y_pred = vot_clf.predict(X_test)

In [ ]:
y_pred.shape

(418,)

In [ ]:
import pandas as pd

data = {"PassengerId" : np.arange(892, 1310),
        "Survived" : y_pred}
data = pd.DataFrame(data=data, index=None)

data

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
result = data.to_csv("result.csv", index=False)

In [ ]:
from google.colab import files
export = files.download("result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>